In [1]:
#修改in_total表无id企业名

#com_origin：404企业原名
#com_search: 正确企业名

from DBUtils.PooledDB import PooledDB
import pymysql

_dbservice_pool = None

class MysqlPool(object): #设置数据库连接池和初始化    
    """
        MYSQL数据库对象，负责产生数据库连接 , 此类中的连接采用连接池实现
        获取连接对象：conn = Mysql.get_connection()
        释放连接对象;conn.close()或del conn
    """
    
    def __init__(self,mincached=5, maxcached=44,
                maxconnections=512, blocking=True, maxshared=0):
        """
            生成MySQL数据库连接池
        :param mincached: 最少的空闲连接数，如果空闲连接数小于这个数，pool会创建一个新的连接
        :param maxcached: 最大的空闲连接数，如果空闲连接数大于这个数，pool会关闭空闲连接
        :param maxconnections: 最大的连接数
        :param blocking: 当连接数达到最大的连接数时，在请求连接的时候，如果这个值是True，请求连接的程序会一直等待，
                          直到当前连接数小于最大连接数，如果这个值是False，会报错，
        :param maxshared: 当连接数达到这个数，新请求的连接会分享已经分配出去的连接
        """        
        db_config = {
            "host": '39.105.9.20',
            "port": 3306,
            "user": 'root',
            "passwd": 'bigdata_oil',
            "db": 'cxd_test',
            "charset": 'utf8'
        }
        self.pool = PooledDB(pymysql, mincached=mincached, maxcached=maxcached, maxconnections=maxconnections,
                             blocking=blocking, maxshared=maxshared, **db_config)

    def get_connection(self):
        return self.pool.connection()

    def close(self):
        self.pool.close()

    def __del__(self):
        self.close()

# 获取mysql数据库服务器的连接
def get_dbservice_mysql_conn(): #实例化连接池类，方便调用
    """
    :return: Object  MySQL Connection
    """
    global _dbservice_pool
    if not _dbservice_pool or not isinstance(_dbservice_pool, MysqlPool):
        _dbservice_pool = MysqlPool()
    return _dbservice_pool.get_connection()

connection = get_dbservice_mysql_conn()


# conn = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')
# conn = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_test',charset = 'utf8')

com_search = []

def update_in_total_name(com_origin, com_correct):
#     connection = conn
    with connection.cursor() as cursor:
        sql = "update `in_total` set `com_name`='{0}' where com_name='{1}'".format(com_correct,com_origin)
        cursor.execute(sql)
    connection.commit()
    connection.close()

# com_search.append(name)

# connection = conn
sql = "SELECT `com_id`,`com_name` FROM `in_total` WHERE `com_id` is null"
with connection.cursor() as cursor:
    cursor.execute(sql)
    result = cursor.fetchall()
com_origin = [com[1] for com in result]

for com_name in com_origin:
    if com_name.endswith('科目余额表'):
        name = com_name[:-5]
        update_in_total_name(com_name, name)
    elif com_name.endswith('序时账'):
        name = com_name[:-3]
        update_in_total_name(com_name, name)
    elif com_name.endswith('19'):
        name = com_name[:-2]
        update_in_total_name(com_name, name)
    else: 
        name = com_name
    com_search.append(name)
            
# def get_com_id(name):
#     connection = pymysql.connect(host = '39.105.9.20',user = 'root',password = 'bigdata_oil',db = 'cxd_data',charset = 'utf8')

# #     name = '大连大昌石油化工有限公司'

#     sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` ='%s'" %name
#     with connection.cursor() as cursor:
#         cursor.execute(sql)
#         result = cursor.fetchone()
#     return result[0]

def update_in_total_id(com_name):
#     print(com_name)
#     connection = conn
    with connection.cursor() as cursor:
        sql = "SELECT `id`,`name` FROM `dic_company` WHERE `name` ='%s'" %com_name
        print(sql)
        cursor.execute(sql)
        result = cursor.fetchone()
    com_id = result[0]
    with connection.cursor() as cursor:
        sql = "update `in_total` set `com_id`='{0}' where com_name='{1}'".format(com_id,com_name)
        print(sql)
        cursor.execute(sql)
    connection.commit()
    connection.close()

for com in com_search:
    if len(com)<4:
        continue
    connection = get_dbservice_mysql_conn()
    update_in_total_id(com)
print('Done')

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='福建省武夷山市竹柴炭公司好美家快捷酒店'
update `in_total` set `com_id`='137257' where com_name='福建省武夷山市竹柴炭公司好美家快捷酒店'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='郯城县交通宾馆有限公司'
update `in_total` set `com_id`='131287' where com_name='郯城县交通宾馆有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连安盾安全咨询有限公司'
update `in_total` set `com_id`='131330' where com_name='大连安盾安全咨询有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='湖北省农村党员干部培训示范基地（夷陵区）稻花香宾馆'
update `in_total` set `com_id`='131286' where com_name='湖北省农村党员干部培训示范基地（夷陵区）稻花香宾馆'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='菏泽盛丰酒店管理有限公司'
update `in_total` set `com_id`='131284' where com_name='菏泽盛丰酒店管理有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='通辽市科尔沁区平安路家悦时尚宾馆'
update `in_total` set `com_id`='130605' where com_name='通辽市科尔沁区平安路家悦时尚宾馆'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='成都信德房产有限公司鼓楼智选假日酒店'
update `in_total` set `com_id`='121310' where com_name='成都信

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油东胜石油（大连）有限公司'
update `in_total` set `com_id`='2618' where com_name='中油东胜石油（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油中广石油化工（大连）有限公司'
update `in_total` set `com_id`='121143' where com_name='中油中广石油化工（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油中庆石油化工（大连）有限公司'
update `in_total` set `com_id`='2738' where com_name='中油中庆石油化工（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油中庆石油化工（大连）有限公司'
update `in_total` set `com_id`='2738' where com_name='中油中庆石油化工（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油中昌（大连）石油化工有限公司'
update `in_total` set `com_id`='121673' where com_name='中油中昌（大连）石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油中昌（大连）石油化工有限公司'
update `in_total` set `com_id`='121673' where com_name='中油中昌（大连）石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油中林石化（大连）有限公司'
update `in_total` set `com_id`='121448' where com_name='中油中林石化（大连）有限公司'

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油千溪能源（大连）有限责任公司'
update `in_total` set `com_id`='115893' where com_name='中油千溪能源（大连）有限责任公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油千溪能源（大连）有限责任公司'
update `in_total` set `com_id`='115893' where com_name='中油千溪能源（大连）有限责任公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油华东石油（大连）有限公司'
update `in_total` set `com_id`='122183' where com_name='中油华东石油（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油华东石油（大连）有限公司'
update `in_total` set `com_id`='122183' where com_name='中油华东石油（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油华中能源（大连）有限公司'
update `in_total` set `com_id`='110982' where com_name='中油华中能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油华中能源（大连）有限公司'
update `in_total` set `com_id`='110982' where com_name='中油华中能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油华南石油销售（大连）有限公司'
update `in_total` set `com_id`='2984' where com_name='中油华南石油销售（大连）有限公司'
SEL

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油宏鑫（大连）石油化工有限公司'
update `in_total` set `com_id`='133413' where com_name='中油宏鑫（大连）石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油宏鑫（大连）石油化工有限公司'
update `in_total` set `com_id`='133413' where com_name='中油宏鑫（大连）石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油富华（大连）石化有限公司'
update `in_total` set `com_id`='2936' where com_name='中油富华（大连）石化有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油富华（大连）石化有限公司'
update `in_total` set `com_id`='2936' where com_name='中油富华（大连）石化有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油巨龙石油化工（大连）有限公司'
update `in_total` set `com_id`='122055' where com_name='中油巨龙石油化工（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油平顺石化（大连）有限公司'
update `in_total` set `com_id`='3096' where com_name='中油平顺石化（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油平顺石化（大连）有限公司'
update `in_total` set `com_id`='3096' where com_name='中油平顺石化（大连）有限公司'
SELECT `i

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油泰和（大连）石油天然气有限公司'
update `in_total` set `com_id`='121994' where com_name='中油泰和（大连）石油天然气有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油泰和（大连）石油天然气有限公司'
update `in_total` set `com_id`='121994' where com_name='中油泰和（大连）石油天然气有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油浦鸿石油（大连）有限公司'
update `in_total` set `com_id`='121633' where com_name='中油浦鸿石油（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油浩博石油化工（大连）有限公司'
update `in_total` set `com_id`='133419' where com_name='中油浩博石油化工（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油浩琦（大连）石油化工有限公司'
update `in_total` set `com_id`='4481' where com_name='中油浩琦（大连）石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油浩琦（大连）石油化工有限公司'
update `in_total` set `com_id`='4481' where com_name='中油浩琦（大连）石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油海元（大连）能源有限公司'
update `in_total` set `com_id`='2296' where com_name='中油海元（大连）能源有

...

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连百恒化工有限公司'
update `in_total` set `com_id`='2758' where com_name='大连百恒化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连益华石油化工有限公司'
update `in_total` set `com_id`='129617' where com_name='大连益华石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连益华石油化工有限公司'
update `in_total` set `com_id`='129617' where com_name='大连益华石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连盛世金源石油化工有限公司'
update `in_total` set `com_id`='2953' where com_name='大连盛世金源石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连盛世金源石油化工有限公司'
update `in_total` set `com_id`='2953' where com_name='大连盛世金源石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连盛泰石油化工有限公司'
update `in_total` set `com_id`='2960' where com_name='大连盛泰石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连盛泰石油化工有限公司'
update `in_total` set `com_id`='2960' where com_name='大连盛泰石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `na

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连长兴龙宇石油化工有限公司'
update `in_total` set `com_id`='2454' where com_name='大连长兴龙宇石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连长兴龙宇石油化工有限公司'
update `in_total` set `com_id`='2454' where com_name='大连长兴龙宇石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连长德能源有限公司'
update `in_total` set `com_id`='121941' where com_name='大连长德能源有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连长恒利石油化工有限公司'
update `in_total` set `com_id`='111053' where com_name='大连长恒利石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连长恒利石油化工有限公司'
update `in_total` set `com_id`='111053' where com_name='大连长恒利石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连陆海航石油化工有限公司'
update `in_total` set `com_id`='3137' where com_name='大连陆海航石油化工有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连陆海航石油化工有限公司'
update `in_total` set `com_id`='3137' where com_name='大连陆海航石油化工有限公司'
SELECT `id`,`name` FROM `dic_company`

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='维多能源（大连）有限公司'
update `in_total` set `com_id`='121313' where com_name='维多能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='美星（大连）石化有限公司'
update `in_total` set `com_id`='3346' where com_name='美星（大连）石化有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='翰源能源（大连）有限公司'
update `in_total` set `com_id`='2721' where com_name='翰源能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='翰源能源（大连）有限公司'
update `in_total` set `com_id`='2721' where com_name='翰源能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='蒙兴石化（大连）有限公司'
update `in_total` set `com_id`='3044' where com_name='蒙兴石化（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='蒙子骄（大连）能源有限公司'
update `in_total` set `com_id`='111064' where com_name='蒙子骄（大连）能源有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='蒙子骄（大连）能源有限公司'
update `in_total` set `com_id`='111064' where com_name='蒙子骄（大连）能源有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连鸿泰佰仕实业发展有限公司'
update `in_total` set `com_id`='121972' where com_name='大连鸿泰佰仕实业发展有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='海润能源（大连）有限公司'
update `in_total` set `com_id`='2140' where com_name='海润能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='海润能源（大连）有限公司'
update `in_total` set `com_id`='2140' where com_name='海润能源（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='海邦石油化工（大连）有限公司'
update `in_total` set `com_id`='3106' where com_name='海邦石油化工（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='盛源石化（大连）有限公司'
update `in_total` set `com_id`='122019' where com_name='盛源石化（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='盛源石化（大连）有限公司'
update `in_total` set `com_id`='122019' where com_name='盛源石化（大连）有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='中油万通石化（大连）有限公司'
update `in_total` set `com_id`='2832' where com_name='中油万通石化（大连）有限公司'
SELECT `id`,`name` FROM `dic_company`

SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连世纪传铭能源有限公司'
update `in_total` set `com_id`='2670' where com_name='大连世纪传铭能源有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连冰淼国际贸易有限公司'
update `in_total` set `com_id`='133479' where com_name='大连冰淼国际贸易有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连冰淼国际贸易有限公司'
update `in_total` set `com_id`='133479' where com_name='大连冰淼国际贸易有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连国金石油有限公司'
update `in_total` set `com_id`='118751' where com_name='大连国金石油有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连国金石油有限公司'
update `in_total` set `com_id`='118751' where com_name='大连国金石油有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连壹点能源有限公司'
update `in_total` set `com_id`='121969' where com_name='大连壹点能源有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连市厚德能源有限公司'
update `in_total` set `com_id`='133480' where com_name='大连市厚德能源有限公司'
SELECT `id`,`name` FROM `dic_company` WHERE `name` ='大连市厚